In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas de evaluación
from sklearn.metrics import accuracy_score


# Para guardar el modelo
import pickle

In [2]:
df = pd.read_csv('./data/titanic_procesado.csv')

In [3]:
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,1.0,0.433152,0.125,0.0,0.368146,1.0
1,0.0,0.0,0.579431,0.125,0.0,0.615097,0.0
2,1.0,0.0,0.462346,0.000,0.0,0.438286,1.0
3,0.0,0.0,0.563806,0.125,0.0,0.595112,1.0
4,1.0,1.0,0.563806,0.000,0.0,0.448347,1.0


In [4]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = X_train.values  # Convertir a NumPy array
y_train = y_train.values  # Convertir a NumPy array
X_test = X_test.values    # Convertir a NumPy array
y_test = y_test.values    # Convertir a NumPy array

In [8]:
# Definir los modelos y sus respectivos hiperparámetros para GridSearch
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'splitter': ['best', 'random'],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3],
            'learning_rate': [0.1, 0.2, 0.3],
            'verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'alpha': [0.1, 1.0, 10.0]
        }
    }
}

# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    # Inicializar GridSearchCV con el modelo y los hiperparámetros
    grid_search = GridSearchCV(
        estimator=info_modelo['modelo'],
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    if nombre == 'Clasificador LGBM':
        # LGBM necesita DataFrame con nombres de columnas
        grid_search.fit(pd.DataFrame(X_train, columns=X.columns), y_train)
        y_pred = grid_search.predict(pd.DataFrame(X_test, columns=X.columns))
    else:
        # Los demás modelos funcionan con NumPy arrays
        grid_search.fit(X_train, y_train)
        y_pred = grid_search.predict(X_test)
    
    # Calcular la precisión de las predicciones
    precision = accuracy_score(y_test, y_pred)
    
    # Almacenar los resultados del modelo
    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })

    estimadores[nombre] = grid_search.best_estimator_
    
    # Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_

# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

Rendimiento de los modelos de clasificación
                                 Modelo  Precisión
4     Clasificador de Gradient Boosting       0.84
6      Clasificador K-Nearest Neighbors       0.83
7                  Clasificador XGBoost       0.82
8                     Clasificador LGBM       0.81
3    Clasificador de Bosques Aleatorios       0.80
1   Clasificador de Vectores de Soporte       0.80
2     Clasificador de Árbol de Decisión       0.80
0                   Regresión Logística       0.79
5                 Clasificador AdaBoost       0.79
10             Clasificador Naive Bayes       0.79
9                            GaussianNB       0.76
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador de Gradient Boosting
Precisión: 0.84


In [9]:
# sin GridSearch 
# Esto ya lo tenemos importado. Lo ponemos nuevamente nada más de referencia
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Creamos el modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluamos el modelo usando precisión
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.80


In [10]:
model = LogisticRegression(
    C=0.5,                  # Valor de regularización
    penalty='l2',            # Tipo de penalización (l2 es la regularización Ridge)
    solver='lbfgs',         # Algoritmo de optimización
    max_iter=200,           # Número máximo de iteraciones
    class_weight='balanced' # Ajustar pesos de las clases
)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.78


In [11]:
# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

In [13]:
# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    grid_search = GridSearchCV(
    estimator=info_modelo['modelo'],
    param_grid=info_modelo['parametros'],
    cv=5,
    scoring='accuracy',
    verbose=0,
    n_jobs=-1,
)

In [14]:
# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Hacer predicciones con el modelo ajustado
y_pred = grid_search.predict(X_test)

# Calcular la precisión de las predicciones
precision = accuracy_score(y_test, y_pred)

# Almacenar los resultados del modelo
puntajes_modelos.append({
    'Modelo': nombre,
    'Precisión': precision
})

estimadores[nombre] = grid_search.best_estimator_

In [15]:
# Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
if precision > mejor_precision:
    mejor_modelo = nombre
    mejor_precision = precision
    mejor_estimador = grid_search.best_estimator_

In [16]:
# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

Rendimiento de los modelos de clasificación
                     Modelo  Precisión
0  Clasificador Naive Bayes       0.79
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador Naive Bayes
Precisión: 0.79


In [17]:
mejor_estimador

,alpha,1.0
,force_alpha,True
,binarize,0.0
,fit_prior,True
,class_prior,None


In [18]:
X_train[0]

array([0.        , 1.        , 0.61581699, 0.        , 0.        ,
       0.55559921, 1.        ])

In [19]:
y_train[0]

np.int64(0)

In [20]:
nuevos_datos = np.array([0,1,0.6159084,0,0,0.55547282,1]).reshape(1,-1)

In [21]:
mejor_estimador.predict(nuevos_datos)

array([0])

In [22]:
with open('modelo.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)